### Tratamento Story Points e Descrição.

In [1]:
import pandas as pd
import re
import emoji
import mysql.connector
from bs4 import BeautifulSoup

In [6]:
# Função para limpar texto preservando código
def clean_text_preserving_tags(text):
    """ 
    Limpa texto removendo ruído, mas preservando tags XML/HTML para garantir a estrutura.
    """
    if pd.isna(text):
        return ""

    # Remover emojis que podem interferir no modelo
    text = emoji.replace_emoji(text, replace='')

    # Remover apenas tags HTML que não carregam significado (ex.: <b>, <i>), mas manter XML/HTML úteis
    text = re.sub(r'</?(?:b|i|p|span|div|br|strong|em)>', '', text)

    # Manter estrutura de código markdown (blocos ``` e ~~~)
    text = re.sub(r'(```.*?```|~~~.*?~~~)', r'\1', text, flags=re.DOTALL)

    # Manter inline code entre <code>...</code>
    text = re.sub(r'<code>(.*?)</code>', r' `\1` ', text)

    # Manter todas as outras tags XML/HTML, incluindo <localStatus>IN_SERVICE</localStatus>
    text = re.sub(r'(<[^>]+>)', r' \1 ', text)  # Adiciona espaço ao redor para facilitar NLP

    # Substituir ponto e vírgula por dois-pontos para evitar problemas de separação no CSV
    text = text.replace(";", ":")

    # Manter estrutura de código e caracteres relevantes para análise
    text = re.sub(r"[^a-zA-Z0-9\s.,!?_(){}\[\]<>:;=#/\*\+\-]", "", text)

    # Substituir múltiplos espaços por um único, mas manter estrutura XML
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Função para aplicar a Normalização SP 3
def normalize_sp3(df):
    """ Aplica a Normalização SP 3 no dataset de issues. """
    
    # Aplicar filtros nos Story Points
    df_filtered = df[
        (df["Story_Point"].notnull()) & 
        (df["Story_Point"] >= 0) & (df["Story_Point"] <= 100) & 
        ((df["Story_Point"] % 1 == 0) | (df["Story_Point"] == 0.5))
    ]

    # Contar total de issues antes dos filtros por projeto
    df_total_before = df.groupby(["Project_ID", "Project_Name"])["Issue_ID"].count().reset_index()
    df_total_before.rename(columns={"Issue_ID": "Total_Issues_Before"}, inplace=True)

    # Remover issues com frequência <= 5 por projeto
    issue_counts = df_filtered.groupby(["Project_ID", "Story_Point"]).Issue_ID.transform("count")
    df_filtered = df_filtered[issue_counts > 5]

    # Contar total de issues após os filtros por projeto
    df_summary = df_filtered.groupby(["Project_ID", "Project_Name"]).agg(
        Total_Issues_After=("Issue_ID", "count")
    ).reset_index()

    # Mesclar para calcular impacto da remoção
    df_final = df_total_before.merge(df_summary, on=["Project_ID", "Project_Name"], how="right")
    df_final["Porcentagem_Removida"] = ((df_final["Total_Issues_Before"] - df_final["Total_Issues_After"]) / df_final["Total_Issues_Before"]) * 100

    # Remover projetos com menos de 100 issues ou impacto > 10%
    df_final = df_final[(df_final["Total_Issues_After"] >= 100) & (df_final["Porcentagem_Removida"] <= 10)]

    # Filtrar novamente as issues que pertencem aos projetos que sobraram
    df_filtered = df_filtered[df_filtered["Project_ID"].isin(df_final["Project_ID"])]

    # Aplicar limpeza preservando código
    df_filtered["Title"] = df_filtered["Title"].apply(clean_text_preserving_tags)
    df_filtered["Description"] = df_filtered["Description"].apply(clean_text_preserving_tags)

    return df_filtered

# Conectar ao MySQL e buscar os dados
def fetch_issues_from_db():
    """ Conecta ao MySQL, executa a consulta e retorna um DataFrame com os dados. """
    
    # Configuração do MySQL (substitua com suas credenciais)
    conn = mysql.connector.connect(
        host="tawos-db",
        user="root",
        password="rootpassword",
        database="tawosdb"
    )

    query = """
    SELECT 
        r.ID AS Repository_ID,
        r.Name AS Repository_Name,
        p.ID AS Project_ID, 
        p.Name AS Project_Name,
        i.ID AS Issue_ID,
        i.Title,
        i.Description,
        i.Assignee_ID,
        i.Story_Point
    FROM Issue i
    JOIN Project p ON i.Project_ID = p.ID
    JOIN Repository r ON p.Repository_ID = r.ID
    WHERE i.Story_Point BETWEEN 0 AND 100
        AND i.Resolution_Date IS NOT NULL
        AND LOWER(TRIM(i.Resolution)) IN ('fixed', 'done', 'completed')
        AND LOWER(TRIM(i.Status)) IN ('closed', 'done', 'resolved', 'complete')
        AND i.Assignee_ID IS NOT NULL
        AND TIMESTAMPDIFF(DAY, i.Creation_Date, i.Resolution_Date) BETWEEN 0 AND 180
        AND i.Description IS NOT NULL AND TRIM(i.Description) <> ''
    """

    # Executar a consulta e carregar os dados
    df = pd.read_sql(query, conn)

    # Fechar conexão
    conn.close()

    return df

# Executar pipeline completo
if __name__ == "__main__":
    print("Buscando dados do banco...")
    df_issues = fetch_issues_from_db()

    print(f"{len(df_issues)} issues carregadas. Aplicando Normalização SP 3...")
    df_normalized = normalize_sp3(df_issues)

    print(f" {len(df_normalized)} issues restantes após normalização.")
    
    # Salvar o resultado em um arquivo CSV
    df_normalized.to_csv("issues_sp_filtered.csv", index=False)
    df_issues.to_csv("issues_sp_unfiltered.csv", index=False)

    print(" Dados normalizados salvos em 'issues_sp_filtered.csv'.")


Buscando dados do banco...


/tmp/ipykernel_126269/1041495142.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


34691 issues carregadas. Aplicando Normalização SP 3...
 20757 issues restantes após normalização.
 Dados normalizados salvos em 'issues_sp_filtered.csv'.


### Criando Developers_ars

In [3]:
import pandas as pd
import mysql.connector

def fetch_issue_start_dates():
    """Busca a data mais próxima do início do desenvolvimento de cada issue e calcula o número de issues resolvidas antes dessa data."""
    
    # Conectar ao banco de dados
    conn = mysql.connector.connect(
        host="tawos-db",
        user="root",
        password="rootpassword",
        database="tawosdb"
    )

    query = """
        SELECT 
            i.ID AS Issue_ID,
            i.Creation_Date,
            i.Creator_ID,
            COALESCE(
                MIN(cl.Creation_Date),  -- Data da primeira mudança para "To Do", "In Progress" ou "To Develop"
                s.First_Sprint_Activated_Date, -- Data de ativação da primeira Sprint associada à issue
                i.Creation_Date -- Caso não tenha nenhuma das anteriores, usa a data de criação da issue
            ) AS In_Progress_Date,
            i.Resolution_Date,
            i.Assignee_ID
        FROM Issue i
        LEFT JOIN Change_Log cl 
            ON cl.Issue_ID = i.ID 
            AND LOWER(TRIM(cl.Field)) = 'status'
            AND LOWER(TRIM(cl.To_String)) IN ('to do', 'in progress', 'to develop')
        LEFT JOIN (
            SELECT 
                il.Sprint_ID, 
                il.ID AS Issue_ID,
                MIN(s.Activated_Date) AS First_Sprint_Activated_Date
            FROM Issue il
            JOIN Sprint s ON il.Sprint_ID = s.ID
            WHERE s.Activated_Date IS NOT NULL
            GROUP BY il.ID
        ) s ON s.Issue_ID = i.ID
        WHERE i.Story_Point BETWEEN 0 AND 100
            AND i.Resolution_Date IS NOT NULL
            AND LOWER(TRIM(i.Resolution)) IN ('fixed', 'done', 'completed')
            AND LOWER(TRIM(i.Status)) IN ('closed', 'done', 'resolved', 'complete')
            AND TIMESTAMPDIFF(DAY, i.Creation_Date, i.Resolution_Date) BETWEEN 0 AND 180
            AND i.Description IS NOT NULL
        GROUP BY i.ID, s.First_Sprint_Activated_Date, i.Creation_Date;
    """

    df = pd.read_sql(query, conn)
    conn.close()
    
    return df

def compute_developer_ar(df, filtered_issues):
    """Calcula quantas issues o desenvolvedor resolveu antes da issue atual, garantindo ordenação correta por resolução.
       Considera apenas issues que estejam no subconjunto de `filtered_issues`.
    """
    
    df["In_Progress_Date"] = pd.to_datetime(df["In_Progress_Date"])
    df["Resolution_Date"] = pd.to_datetime(df["Resolution_Date"])

    # Filtrar apenas as issues do subconjunto do CSV, mantendo apenas as colunas necessárias
    df_filtered = df[df["Issue_ID"].isin(filtered_issues["Issue_ID"])].copy()

    developer_ars = {}

    for assignee in df_filtered["Assignee_ID"].unique():
        df_dev = df_filtered[df_filtered["Assignee_ID"] == assignee].sort_values("Resolution_Date")

        resolved_counts = []
        for i, row in df_dev.iterrows():
            # Contar apenas issues resolvidas antes da atual
            count = df_dev[
                (df_dev["Resolution_Date"] < row["In_Progress_Date"]) & 
                (df_dev["Resolution_Date"] >= row["In_Progress_Date"] - pd.DateOffset(years=1))
            ].shape[0]
            resolved_counts.append(count)

        developer_ars.update(dict(zip(df_dev["Issue_ID"], resolved_counts)))

    df_filtered.loc[:, "Developer_AR"] = df_filtered["Issue_ID"].map(developer_ars)

    # Selecionar apenas as colunas relevantes
    return df_filtered[["Issue_ID", "In_Progress_Date", "Resolution_Date", "Creation_Date", "Creator_ID", "Developer_AR"]]

if __name__ == "__main__":
    print("Carregando subconjunto de issues...")
    df_filtered_issues = pd.read_csv("issues_sp_filtered.csv")

    print("Buscando datas de início das issues...")
    df_issues = fetch_issue_start_dates()

    print("Calculando número de issues resolvidas antes de cada issue...")
    df_issues_filtered = compute_developer_ar(df_issues, df_filtered_issues)

    print("Fazendo merge com o CSV original...")
    df_final = df_filtered_issues.merge(df_issues_filtered, on="Issue_ID", how="left")

    print("Visualizando os primeiros resultados:")
    print(df_final.head())

    # Salvar para análise posterior
    df_final.to_csv("issues_sp_filtered_with_developer_ar.csv", index=False)
    print("Arquivo salvo com sucesso: issues_sp_filtered_with_developer_ar.csv")
    

Carregando subconjunto de issues...
Buscando datas de início das issues...


/tmp/ipykernel_285833/71804283.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Calculando número de issues resolvidas antes de cada issue...
Fazendo merge com o CSV original...
Visualizando os primeiros resultados:
   Repository_ID Repository_Name  Project_ID    Project_Name  Issue_ID  \
0              2        Sonatype           3  Sonatype Nexus      5004   
1              2        Sonatype           3  Sonatype Nexus      5049   
2              2        Sonatype           3  Sonatype Nexus      5129   
3              2        Sonatype           3  Sonatype Nexus      5203   
4              2        Sonatype           3  Sonatype Nexus      5268   

                                               Title  \
0  Conan integration in 3.22.0 does not handle He...   
1  Problem proxying NuGet packages hosted by GitH...   
2  NullPointer appears if run cleanup for hosted ...   
3             [R format] regression test full format   
4  [Helm Features] Cleanup policy facet should be...   

                                         Description  Assignee_ID  \
0  I am unabl

# Calculo da Reputação

In [4]:
import pandas as pd
from datetime import timedelta

def calculate_temporal_reputation(df):
    """Calcula reputação temporal usando resolução recente (últimos 12 meses) como critério de janela."""
    df = df.copy()
    df["In_Progress_Date"] = pd.to_datetime(df["In_Progress_Date"])
    df["Resolution_Date"] = pd.to_datetime(df["Resolution_Date"])

    reputations = []

    for i, row in df.iterrows():
        assignee = row["Assignee_ID"]
        in_progress_date = row["In_Progress_Date"]
        start_window = in_progress_date - timedelta(days=365)

        # Janela de issues resolvidas nos 12 meses anteriores
        window_df = df[
            (df["Resolution_Date"] >= start_window) &
            (df["Resolution_Date"] < in_progress_date)
        ]

        # Issues criadas por esse desenvolvedor
        opened = window_df[window_df["Creator_ID"] == assignee]

        # Destas, quantas ele também resolveu
        opened_and_fixed = opened[opened["Assignee_ID"] == assignee]

        reputation_t = len(opened_and_fixed) / (len(opened) + 1)
        reputations.append(reputation_t)

    df["Reputation_t"] = reputations
    return df

if __name__ == "__main__":
    print("Carregando CSV com Developer_AR...")
    df = pd.read_csv("issues_sp_filtered_with_developer_ar.csv")

    print("Calculando reputação temporal com base em Resolution_Date (últimos 12 meses)...")
    df_result = calculate_temporal_reputation(df)

    print("Amostra dos resultados:")
    print(df_result[["Issue_ID", "Assignee_ID", "Developer_AR", "Reputation_t"]].head())

    df_result.to_csv("issues_sp_filtered_with_reputation_t.csv", index=False)
    print("Arquivo final salvo: issues_sp_filtered_with_reputation_t.csv")


Carregando CSV com Developer_AR...
Calculando reputação temporal com base em Resolution_Date (últimos 12 meses)...
Amostra dos resultados:
   Issue_ID  Assignee_ID  Developer_AR  Reputation_t
0      5004          518             1           0.5
1      5049          462             0           0.0
2      5129          500             1           0.0
3      5203          500             0           0.0
4      5268          518             0           0.0
Arquivo final salvo: issues_sp_filtered_with_reputation_t.csv


In [5]:
import pandas as pd
import mysql.connector

def fetch_issue_type_and_comments(issue_ids):
    """Consulta o banco para retornar tipo da issue e número de comentários por issue + assignee."""
    conn = mysql.connector.connect(
        host="tawos-db",
        user="root",
        password="rootpassword",
        database="tawosdb"
    )

    # Converter lista para string de IDs
    issue_id_list = ",".join(map(str, issue_ids))

    query = f"""
        SELECT 
            i.ID AS Issue_ID,
            i.Type AS Issue_Type,
            COUNT(c_all.ID) AS Num_Comentarios,
            COUNT(CASE WHEN c_all.Author_ID = i.Assignee_ID THEN 1 END) AS Comentarios_Assignee
        FROM Issue i
        LEFT JOIN Comment c_all ON c_all.Issue_ID = i.ID
        WHERE i.ID IN ({issue_id_list})
        GROUP BY i.ID
    """

    df = pd.read_sql(query, conn)
    conn.close()
    return df

if __name__ == "__main__":
    print("Carregando CSV base...")
    df_base = pd.read_csv("issues_sp_filtered_with_reputation_t.csv")

    print("Buscando dados do banco...")
    issue_ids = df_base["Issue_ID"].tolist()
    df_comments = fetch_issue_type_and_comments(issue_ids)

    print("Mesclando dados com CSV...")
    df_merged = df_base.merge(df_comments, on="Issue_ID", how="left")

    print("Visualizando amostra:")
    print(df_merged[["Issue_ID", "Issue_Type", "Num_Comentarios", "Comentarios_Assignee"]].head())

    df_merged.to_csv("issues_sp_filtered_with_comments.csv", index=False)
    print("Novo CSV salvo: issues_sp_filtered_with_comments.csv")


Carregando CSV base...
Buscando dados do banco...


/tmp/ipykernel_285833/3069545764.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Mesclando dados com CSV...
Visualizando amostra:
   Issue_ID Issue_Type  Num_Comentarios  Comentarios_Assignee
0      5004        Bug                4                     0
1      5049        Bug                0                     0
2      5129        Bug                0                     0
3      5203       Task                0                     0
4      5268      Story                5                     3
Novo CSV salvo: issues_sp_filtered_with_comments.csv


In [ ]:
import pandas as pd

# Carregar o CSV existente
df = pd.read_csv("issues_sp_filtered_with_comments.csv")

# Calcular estatísticas por projeto
project_stats = df.groupby("Project_ID")["Story_Point"].agg(
    SP_Intervalo_Projeto=lambda x: x.max() - x.min(),
    Project_SP_Std="std"
).reset_index()

# Mesclar com o DataFrame original
df_enriched = df.merge(project_stats, on="Project_ID", how="left")

# Visualizar amostra
print(df_enriched[["Project_ID", "Story_Point", "SP_Intervalo_Projeto", "Project_SP_Std"]].head())

# Salvar novo CSV
df_enriched.to_csv("issues_sp_with_project_stats.csv", index=False)
print("Novo CSV salvo como 'issues_sp_with_project_stats.csv'")
